<a href="https://colab.research.google.com/github/DCODE-ARMY/GAN/blob/3DCNN/darshan_mainFrame_CNN-RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Tue Aug  2 16:03:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv3D, MaxPooling3D, Dropout, BatchNormalization
from keras import layers
import numpy as np
import matplotlib.pyplot as plt
import h5py
import pickle
import cv2
import collections
from sklearn.utils import shuffle


In [3]:
image_size=256
train_data_dir='/content/drive/MyDrive/crime detection/train'
categories=['arrest', 'arson', 'abuse', 'normal', 'assault']
test_data_dir='/content/drive/MyDrive/crime detection/test'
batch_size=32
temporal_length=30
stride=15
channel_axis=temporal_length
repeat=1500
feature_length=2048


In [4]:
#getting list of videos
import os 
global files_list
temp_files_list=[]



for i in categories:
  temp_files_list.append(os.listdir(train_data_dir+'/{}'.format(i)))

print([len(temp_files_list[i]) for i in range(5)])

#changing files names to directory name


for i in range(len(categories)):

  for k in range(len(temp_files_list[i])):
    temp_files_list[i][k] =train_data_dir+"/{}/{}".format(categories[i],temp_files_list[i][k])

#len of all videos 
print([len(temp_files_list[i]) for i in range(5)])

#changing categories to single list
files_list=[]

for i in range(len(temp_files_list)):
  for j in range(len(temp_files_list[i])):
    files_list.append(temp_files_list[i][j])    

files_list=shuffle(files_list)
len(files_list)


[42, 36, 27, 41, 31]
[42, 36, 27, 41, 31]


177

In [5]:
import os


# getting test videos name
test_files=[]
for i in os.listdir(test_data_dir):
  test_files.append(i)

# getting test video path 
for i in range(len(test_files)):
  test_files[i]=test_data_dir+"/{}".format(test_files[i])

# print(len(test_files))
# test_files



In [6]:
def build_feature_extractor():
    feature_extractor = tf.keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(image_size, image_size, 3),
    )
    inputs = keras.Input((image_size, image_size, 3))
    preprocess_input = keras.applications.inception_v3.preprocess_input(inputs)
    
    outputs = feature_extractor(preprocess_input)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

In [7]:

"""manualy shuffling data for epoch since using custom generator and also the bulit in gen wont shuffle first.
it splits the data first and then at the end of every epoch it shuffles"""


def gen(data=files_list,temporal_length=temporal_length,temporal_stride=stride,batch_size=batch_size):  
  
  batch_count=0
  ch=0 #check var
  frame_features=[]
  frame_masks = []
  labels=[]
  chunk_frames=[]

  for i in range(len(data)):
    frame_count=0 #to check enough frames are available to append in queue
    chunk_frames.clear() # batch list 
    frame_features.clear()

    labels.clear()
    batch_count=0 #when new video starts the batch begins form Zero
  

   
    
    cap=cv2.VideoCapture(data[i])
    total_frames=int(cap.get(7))

    for k in range(total_frames):
      
      if total_frames-frame_count >=temporal_length:

        if len(frame_features) != temporal_length:
          img=cap.read()[1]
          img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
          
          img=img.astype(np.float32)
          img=tf.keras.layers.Resizing(height=256,width=256,interpolation='gaussian')(img)
        
         
          frame_features.append(feature_extractor.predict(np.expand_dims(img,axis=0)).squeeze())
        
          frame_count+=1

        if len(frame_features) == temporal_length:  

          chunk_frames.append(np.array(frame_features))
          labels.append(float(categories.index(data[i].split('_')[0].split('/')[-1])))
          
          
          batch_count+=1
                  
            
        if batch_count == batch_size:
          chunk_frames=shuffle(chunk_frames)  
          batch_return=np.array(chunk_frames)
         
          # labels_return=np.reshape(np.array(labels,ndmin=2),(32,1))     
          labels_return=labels
          frame_masks=np.ones(shape=(batch_size,temporal_length),dtype='bool')  
                  
          yield  (batch_return,frame_masks) , labels_return
          
          batch_count=0
          labels.clear()
          chunk_frames.clear()
          frame_features.clear()
          

          batch_return=0
          labels_return=0


      else:
        batch_count=0
        continue
    cap.release()


gen_data=tf.data.Dataset.from_generator(gen, output_signature=( (tf.TensorSpec(shape=(batch_size, temporal_length,feature_length),dtype=tf.dtypes.bool),tf.TensorSpec(shape=(batch_size, temporal_length),dtype=tf.dtypes.int8)),tf.TensorSpec(shape=(batch_size),dtype=tf.dtypes.float32) ))
gen_data=gen_data.prefetch(buffer_size=tf.data.AUTOTUNE).repeat(repeat)


In [8]:
# #  np.isinf([np.inf, -np.inf, 1.0, np.nan])
# import numpy as np
# print(np.any(np.sum(np.array([7, 5, 1.0, np.nan]))))
# np.isnan(np.sum(np.array([7, 5, 1.0, np.nan])))

In [9]:
def val_gen(val_data=test_files,temporal_lenght=temporal_length,temporal_stride=stride,batch_size=batch_size):  
  chunk_frames=[]
  batch_count=0
  frame_features=[]
  frame_masks = []
  labels=[]
  chunk_frames=[]
 

  # val_data=shuffle(val_data)

  for i in range(len(val_data)):


    frame_count=0 #to check enough frames are available to append in queue
    chunk_frames.clear() # batch list 
    
    frame_features.clear()

    labels.clear()
    batch_count=0 #when new video starts the batch begins form Zero
    temp_frames=[]
    main_frames=[]
    
    

    total_frames=int(cv2.VideoCapture(val_data[i]).get(7))
    cap=cv2.VideoCapture(val_data[i])

    for k in range(total_frames):
      
      
      if total_frames-frame_count >=temporal_lenght:

        if len(frame_features) != temporal_lenght:
          img=cap.read()[1]
          img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
          img=img.astype(np.float32)

          img=tf.keras.layers.Resizing(image_size,image_size,interpolation='gaussian',crop_to_aspect_ratio=False)(img)
          # img = tf.keras.layers.Rescaling(scale=1./255)(img)
          frame_features.append(feature_extractor.predict(np.expand_dims(img,axis=0)).squeeze())
          frame_count+=1

        if len(frame_features) == temporal_lenght:

          chunk_frames.append(np.array(frame_features))
          labels.append(float(categories.index(val_data[i].split('_')[0].split('/')[-1])))
          batch_count+=1
            
        if batch_count == batch_size:
          chunk_frames=shuffle(chunk_frames)  
          batch_return=np.array(chunk_frames)
         
          # labels_return=np.reshape(np.array(labels,ndmin=2),(32,1))     
          labels_return=labels
          frame_masks=np.ones(shape=(batch_size,temporal_length),dtype='bool')
          

          yield  (batch_return,frame_masks) , labels_return
        
          batch_count=0
          labels.clear()
          chunk_frames.clear()
          frame_features.clear()

          batch_return=0
          labels_return=0



      else:
        continue
    cap.release()

val_data=tf.data.Dataset.from_generator(val_gen, output_signature=((tf.TensorSpec(shape=(batch_size, temporal_length, feature_length),dtype=tf.dtypes.float32),tf.TensorSpec(shape=(batch_size, temporal_length),dtype=tf.dtypes.bool)),tf.TensorSpec(shape=(batch_size),dtype=tf.dtypes.float32) ))
val_data=val_data.prefetch(buffer_size=tf.data.AUTOTUNE).repeat(repeat)

In [10]:
def get_sequence_model():

  frame_features_input = keras.Input((temporal_length, feature_length))
  mask_input = keras.Input((temporal_length,), dtype="bool")

  # Refer to the following tutorial to understand the significance of using `mask`:
  # https://keras.io/api/layers/recurrent_layers/gru/
  x = keras.layers.GRU(64, return_sequences=True)(
      frame_features_input, mask=mask_input
  )
  x = keras.layers.GRU(32)(x)
  x = keras.layers.Dropout(0.4)(x)
  x = keras.layers.Dense(512, activation="relu")(x)
  x=keras.layers.BatchNormalization()(x)
  output = keras.layers.Dense(len(categories), activation="softmax")(x)

  rnn_model = keras.Model([frame_features_input, mask_input], output)

  
  return rnn_model



In [11]:
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(['a','b','c','v','a','b']))
print(label_processor.get_vocabulary())
print(np.array(label_processor(['a','b']).numpy()).shape)

['a', 'b', 'c', 'v']
(2,)


/usr/local/lib/python3.7/dist-packages/numpy/core/numeric.py:2446: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


In [ ]:
filepath='/content/drive/MyDrive/crime detection'
checkpoint = keras.callbacks.ModelCheckpoint(filepath, save_weights_only=True, save_best_only=True, verbose=1)


%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
# for running on gpu
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))


class CustomCallback(keras.callbacks.Callback):

  def on_epoch_end(self, epoch, logs=None):
    global files_list
    files_list=shuffle(files_list)

csv_logger=tf.keras.callbacks.CSVLogger('/content/drive/MyDrive/crime detection/log.csv', separator=",", append=True)      
    

# initial_learning_rate = 0.01
# lr_schedule = keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate, decay_steps=1000, decay_rate=0.96, staircase=False
# )

# lr_scheduler=tf.keras.callbacks.LearningRateScheduler(lr_schedule)
# Define callbacks.
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    "/content/drive/MyDrive/crime detection/model.{epoch:02d}-{val_accuracy:.2f}.h5", save_best_only=True,monitor='val_accuracy'
)

adam=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.9, beta_2=0.999, epsilon=1e-07)



with tf.device('/device:GPU:0'):

  model=get_sequence_model()
  model.compile(
      
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      
      optimizer=adam,

      metrics=["accuracy"]  )
  
model.fit(gen_data,validation_data=val_data,validation_steps=48,epochs=1000,steps_per_epoch=100,callbacks=[csv_logger,checkpoint_cb,CustomCallback()],batch_size=batch_size)
  






Found GPU at: /device:GPU:0
Epoch 1/1000
150/150 [==============================] - 474s 3s/step - loss: 1.2238 - accuracy: 0.5529 - val_loss: 1.1094 - val_accuracy: 0.0000e+00
Epoch 2/1000
150/150 [==============================] - 536s 4s/step - loss: 1.4534 - accuracy: 0.4792 - val_loss: 0.4180 - val_accuracy: 1.0000
Epoch 3/1000
150/150 [==============================] - 512s 3s/step - loss: 1.1653 - accuracy: 0.6077 - val_loss: 4.6329 - val_accuracy: 0.0000e+00
Epoch 4/1000
 55/150 [==========>...................] - ETA: 5:29 - loss: 1.2231 - accuracy: 0.6182

In [ ]:
#678  train batches
#49 test batches

In [ ]:
# x_=[]
# for j in  test_files:
#   x_.append(cv2.VideoCapture(j).get(7))
#   # print(j)
# print(x_)
# print(sum(x_))
# print((int(sum(x_))-30)/15,((int(sum(x_))-30)/15)/32)

# 23848.0 #total frames
# 1587.8666666666666 #after made into chunks
# 49.62083333333333 #after put into batches. So totally 49 batches available

In [ ]:
# x_=[]
# for i in range(5):
#   for j in range(len(files_list[i])):
#     x_.append(cv2.VideoCapture(files_list[i][j]).get(7))
# print(x_)
# print(sum(x_))
# print((int(sum(x_))-30)/15,((int(sum(x_))-30)/15)/32)

# (11275-30)/15


In [ ]:
# 256%9

In [ ]:
# from tensorflow.keras.utils import plot_model

# plot_model(model, to_file='convolutional_neural_network.png')

In [ ]:
# for i in range(5):
#   print(files_list[i][0])
#   files_list[i]=shuffle(files_list[i])
#   print(files_list[i][0])

3%3 ==0


In [ ]:
4279/32  #valid 133

1163 # train



In [ ]:
# loss:2.0897 acc:0.2936 val_loss: 4.4075 - val_accuracy: 0.4333 epoch 3
# loss: 1.9303 - accuracy: 0.2906 - val_loss: 8.7073 - val_accuracy: 0.4333 epoch 4

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np

total_frames=int(cv2.VideoCapture('/content/drive/MyDrive/train/abuse/abuse_8.mp4').get(7))
print(total_frames)
tol=0

# img is 2D image data
# tol  is tolerance
cap=cv2.VideoCapture('/content/drive/MyDrive/train/abuse/abuse_8.mp4')
img=cap.read()[1]
# img=cv2.resize(img,(180,180))
cv2_imshow(img)


In [ ]:
y_nonzero, x_nonzero, _ = np.nonzero(im)
im=im[np.min(y_nonzero):np.max(y_nonzero), np.min(x_nonzero):np.max(x_nonzero)]
cv2_imshow(im)

In [ ]:
# to try 
"""
Clip the gradients to prevent their explosion. For instance in Keras you could use clipnorm=1. or clipvalue=1. as parameters for your optimizer.

Try to increase the batch size (e.g. 32 to 64 or 128) to increase the stability of your optimization.

Check the size of your last batch which may be different from the batch size.

Try to increase the batch size (e.g. 32 to 64 or 128) to increase the stability of your optimization.

Check validity of inputs (no NaNs or sometimes 0s). i.e df.isnull().any()
array_sum = np.sum(array)
array_has_nan = np.isnan(array_sum)
"""

In [ ]:
"""Add regularization to add l1 or l2 penalties to the weights. Otherwise, try a smaller l2 reg. i.e l2(0.001), 
      or remove it if already exists.

Try a smaller Dropout rate.

Clip the gradients to prevent their explosion. For instance in Keras you could use clipnorm=1. or clipvalue=1. as parameters for your optimizer.

Check validity of inputs (no NaNs or sometimes 0s). i.e df.isnull().any()

Replace optimizer with Adam which is easier to handle. Sometimes also replacing sgd with rmsprop would help.

Use RMSProp with heavy regularization to prevent gradient explosion.

Try normalizing your data, or inspect your normalization process for any bad values introduced.

Verify that you are using the right activation function (e.g. using a softmax instead of sigmoid for multiple class classification).

Try to increase the batch size (e.g. 32 to 64 or 128) to increase the stability of your optimization.

Check the size of your last batch which may be different from the batch size."""

In [ ]:
# # model.history.history
# import pickle
# # with open('/content/drive/MyDrive/history/trainHistoryDict_2', 'wb') as file_pi:
# #   pickle.dump(model.history.history, file_pi)
# history = pickle.load(open('/content/drive/MyDrive/history/trainHistoryDict_main', "rb"))

In [ ]:
# aaaa=history
# bb=history

# for d in aaaa.keys():
#   bb[d].append(aaaa[d][-1])
  

In [ ]:
# model=tf.keras.models.load_model('/content/drive/MyDrive/re_train/my_model')

In [ ]:
# model.save('/content/drive/MyDrive/re_train/my_model')

In [ ]:
import matplotlib.pyplot as plt
plt.plot(model.history.history['accuracy'])
plt.plot(model.history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(model.history.history['loss'])
plt.plot(model.history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(model.history.history['accuracy'])
plt.plot(model.history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(model.history.history['loss'])
plt.plot(model.history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
#for predicting output

import collections
import copy
import numpy as np
import cv2
from sklearn.utils import shuffle
from keras.utils import np_utils
import tensorflow as tf

def predict(data,temporal_lenght=30,temporal_stride=20,batch_size=32):  
  
  chunk_frames=[]
  batch_count=0
  queue=collections.deque()
  label=collections.deque()
  # model=tf.keras.models.load_model('/content/drive/MyDrive/saved_model/my_model')

  for i in range(len(data)):
    frame_count=0 #to check enough frames are available to append in queue
    chunk_frames.clear() # batch list 
    queue.clear() # frame queue
    label.clear() # lable queue
    batch_count=0 #when new video starts the batch begins form Zero
    lable_list=[0,0,0,0,0]#for labeling 
    
    total_frames=int(cv2.VideoCapture(data[i]).get(7))
    print(total_frames)
    cap=cv2.VideoCapture(data[i])
    for k in range(total_frames):
      
      if total_frames-frame_count >=temporal_lenght:
        if len(queue) != temporal_lenght:
          queue.append(cv2.resize(cap.read()[1],(180,180)))
          #queue.append(cap.read()[1])
          frame_count+=1

        if len(queue) == temporal_lenght:
          
          chunk_frames.append(np.array(queue,ndmin=4))
          for stride in range(temporal_stride):
            queue.popleft()          
          batch_return=np.array(chunk_frames)

          #print(model.predict(batch_return))
                          
          yield  batch_return 
        
        chunk_frames.clear()
        batch_return=0
        



    else:
      batch_count=0
      continue
    cap.release()

In [ ]:
# 'arrest'-0, 'abuse'-1, 'Normal_Videos'-2, 'Arson '-3, 'assault'-4
'Normal_Videos', 'assault', 'arson', ' arrest', 'abuse'
'Normal_Videos', 'assault', 'arson', ' arrest', 'abuse'

In [ ]:
model=tf.keras.models.load_model('/content/drive/MyDrive/high acc/model.11-0.71.h5')
model.compile()

In [ ]:
model.save('/content/drive/MyDrive/new_model')

In [ ]:
# to predict output


dataa=['/content/drive/MyDrive/train/Normal_Videos/Normal_Videos_015_x264.mp4']
pr=predict(data=dataa)
ff=[]
for i in range(1000):
  x=next(pr)
  ff.append(model.predict(x))
  print(model.predict(x))


In [ ]:
# print(np.count(np.array(ff),3))

unique, counts = np.unique(np.array(ff), return_counts=True)
dict(zip(unique, counts))


import matplotlib.pyplot as plt
loss_train = model.history.history['loss']
loss_val = model.history.history['val_loss']
epochs = range(1,3,1)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
import cv2

from google.colab.patches import cv2_imshow
face=cv2.imread("/content/im_1.jfif")
cv2_imshow(face)


In [ ]:
import tensorflow_hub as hub
# import tensorflow as tf

# model = hub.load("https://tfhub.dev/captain-pool/esrgan-tf2/1")
# # To add an extra dimension for batch, use tf.expand_dims()
# low_resolution_image = tf.expand_dims(face, 0) # Low Resolution Image of shape [batch_size, height, width, 3]
# low_resolution_image = tf.cast(low_resolution_image, tf.float32)
# super_resolution = model(low_resolution_image) # Perform Super Resolution here
# print(super_resolution[0].shape)

# cv2_imshow(super_resolution[0].numpy())

In [ ]:
! pip install git+https://github.com/rcmalli/keras-vggface.git

!pip show keras-vggface

!pip install keras_applications
# check version of keras_vggface
import keras_vggface
# print version
print(keras_vggface.__version__)

In [ ]:
!pip install mtcnn
# confirm mtcnn was installed correctly
import mtcnn
# print version
print(mtcnn.__version__)

In [ ]:
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from keras_vggface.utils import decode_predictions

model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')

In [ ]:
model.summary()

In [ ]:
from matplotlib import pyplot
from PIL import Image
from numpy import asarray
from scipy.spatial.distance import cosine
from mtcnn.mtcnn import MTCNN
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
 
# extract a single face from a given photograph
def extract_face(filename,size=(224,224)):
  print(filename)
  pixels= pyplot.imread(filename)
  cv2_imshow(pixels)
  detector = MTCNN()
  results = detector.detect_faces(pixels) 
  x1, y1, width, height = results[0]['box']
  x2, y2 = x1 + width, y1 + height
  face = pixels[y1:y2, x1:x2]
  cv2_imshow(cv2.resize(face,dsize=size))
  image = Image.fromarray(face)
  image = image.resize(size)
 
  face_array = asarray(image)
  return face_array

# extract faces and calculate face embeddings for a list of photo files
def get_embeddings(filenames):
	# extract faces
	faces = [extract_face(f) for f in filenames]
	# convert into an array of samples
	samples = asarray(faces, 'float32')
	# prepare the face for the model, e.g. center pixels
	samples = preprocess_input(samples, version=2)
	# create a vggface model
	model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
	# perform prediction
	yhat = model.predict(samples)
	return yhat
 
# determine if a candidate face is a match for a known face
def is_match(known_embedding, candidate_embedding, thresh=0.5):
	# calculate distance between embeddings
	score = cosine(known_embedding, candidate_embedding)
	if score <= thresh:
		print('>face is a Match (%.3f <= %.3f)' % (score, thresh))
	else:
		print('>face is NOT a Match (%.3f > %.3f)' % (score, thresh))
 
# define filenames
filenames = ['/content/im_1.jfif', '/content/im_2.jpg',
	'/content/im-4.jfif', '/content/im_5.jfif']
# get embeddings file filenames
embeddings = get_embeddings(filenames)
# define sharon stone
sharon_id = embeddings[0]
# verify known photos of sharon
print('Positive Tests')
is_match(embeddings[0], embeddings[1])
is_match(embeddings[0], embeddings[2])
# verify known photos of other people
print('Negative Tests')
is_match(embeddings[0], embeddings[3])